# Library

In [1]:
! pip install wandb opencv-python-headless==4.1.2.30 albumentations torch-summary timm einops joblib icecream  -qq -U

     |████████████████████████████████| 1.7 MB 7.8 MB/s 
     |████████████████████████████████| 21.8 MB 1.1 MB/s 
     |████████████████████████████████| 102 kB 70.7 MB/s 
     |████████████████████████████████| 431 kB 77.4 MB/s 
     |████████████████████████████████| 97 kB 7.9 MB/s 
     |████████████████████████████████| 142 kB 70.0 MB/s 
     |████████████████████████████████| 180 kB 67.5 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 


In [2]:
from sklearn.metrics import f1_score
from glob import glob
import pathlib
from pathlib import Path
from torchsummary import summary
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pprint import pprint
import urllib.request
import csv
import numpy as np
from einops import rearrange, reduce, repeat
from torch.cuda import amp
from tqdm import tqdm
import wandb
import time
import copy
from collections import defaultdict
from sklearn.metrics import mean_squared_error
import joblib
import gc
import os
from icecream import ic
from sklearn.model_selection import train_test_split
import gc
import cv2
import copy
import time
import random
from PIL import Image

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

import timm

import json

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


from sklearn.metrics import f1_score

# ENV

In [3]:

# ENV = 'COLAB'
ENV = 'KAGGLE'
# ENV = 'SYSTEM'

# Option for Mixed Precision
# FP16 = True
FP16 = False


CONFIG = dict(
    nickname = 'b6noisy512 ',
    seed=42,
    backbone='tf_efficientnet_b6_ns',
    embedder=None,
    train_batch_size=4,
    valid_batch_size=16,
    img_size=512,
    num_epochs=50,
    early_stopping = False,
    early_stopping_step = 5,
    learning_rate=1e-4,
    scheduler='CosineAnnealingLR',
    min_lr=1e-6,
    T_max=100,
    num_classes = 25,
    weight_decay=1e-6,
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    competition='lg',
    _wandb_kernel='deb'
)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# SET SEED 

In [4]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)


set_seed(CONFIG['seed'])


# Read the Data


In [5]:
import wandb
run = wandb.init(project="lg", entity="jiwon7258",
                 config=CONFIG, job_type='train', id='2priu350',resume=True)
dataset = wandb.run.use_artifact(
    'jiwon7258/lg/lg_train:v0', type='dataset')


# Download the artifact's contents
dataset_dir = dataset.download()
dataset_dir = Path(dataset_dir)


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Downloading large artifact lg_train:v0, 918.24MB. 17301 files... Done. 0:0:0


In [6]:
TRAIN_PATH = dataset_dir
# TEST_PATH = dataset_dir / 'test'

# Augmentations

In [7]:
data_transforms = {
    "train": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05,
                           rotate_limit=35, p=0.5),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15,
                   b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Normalize(),
        ToTensorV2()], p=1.),

    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(),
        ToTensorV2()], p=1.)
}


# Dataset

In [8]:
train_csv = sorted(glob(str(TRAIN_PATH / '*/*.csv')))
train_jpg = sorted(glob(str(TRAIN_PATH / '*/*.jpg')))
train_json = sorted(glob(str(TRAIN_PATH / '*/*.json')))


crops = []
diseases = []
risks = []
labels = []

for i in range(len(train_json)):
    with open(train_json[i], 'r') as f:
        sample = json.load(f)
        crop = sample['annotations']['crop']
        disease = sample['annotations']['disease']
        risk = sample['annotations']['risk']
        label=f"{crop}_{disease}_{risk}"
    
        crops.append(crop)
        diseases.append(disease)
        risks.append(risk)
        labels.append(label)
        
label_unique = sorted(np.unique(labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in labels] # len = train_len

In [9]:
train_jpg = np.array(train_jpg)
train_labels = np.array(train_labels)

In [10]:
class CustomDataset(Dataset):
    def __init__(self, train_img, train_label, transforms=None):
        self.imgs = train_img
        self.labels = train_label
        self.transforms = transforms
        
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, index):
        img_path = self.imgs[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        target = self.labels[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return img, target
    
# trainDataset = CustomDataset(X_train, y_train, transforms = data_transforms['train'])
# trainDataloader = DataLoader(
#     trainDataset, batch_size=CONFIG['train_batch_size'], shuffle=True)

# validDataset = CustomDataset(X_val, y_val, transforms = data_transforms['valid'])
# validDataloader = DataLoader(validDataset, batch_size = CONFIG['valid_batch_size'], shuffle = True)

In [11]:
from sklearn.model_selection import StratifiedKFold
train_datasets = []
valid_datasets = []
train_dataloaders = []
valid_dataloaders = []

skf = StratifiedKFold(n_splits = 5)

for step, (train_index, val_index) in enumerate(skf.split(X = train_jpg, y= train_labels)):
    X_train = train_jpg[train_index]
    y_train = train_labels[train_index]
    X_val = train_jpg[val_index]
    y_val = train_labels[val_index]
    train_datasets.append(CustomDataset(
        X_train, y_train, transforms=data_transforms['train']))
    valid_datasets.append(CustomDataset(
        X_val, y_val, transforms=data_transforms['valid']))
    train_dataloaders.append(DataLoader(
        train_datasets[step], batch_size=CONFIG['train_batch_size'], shuffle=True)
    )
    valid_dataloaders.append(
        DataLoader(
            valid_datasets[step], batch_size=CONFIG['valid_batch_size'], shuffle=True)
    )


# Model

In [12]:
class Model(nn.Module):
    def __init__(self, backbone, embedder, pretrained=True):
        super(Model, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.n_features = self.backbone.num_features
        self.fc = nn.Linear(self.n_features, CONFIG['num_classes'])

    def forward(self, images):
        # features = (bs, embedding_size)
        features = self.backbone(images)
        # outputs  = (bs, num_classes)
        # if isinstance(features, tuple):
        #   features = features[0]
        output = self.fc(features)
        return output


model = Model(CONFIG['backbone'], CONFIG['embedder'], pretrained = True)
model.to(CONFIG['device'])
;

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b6_ns-51548356.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b6_ns-51548356.pth


''

In [13]:
optimizer = torch.optim.Adam(
    params=model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])


In [14]:
def criterion(logits: torch.tensor, targets: torch.tensor):
    return nn.CrossEntropyLoss()(logits.view(-1,CONFIG['num_classes']), targets.view(-1))

   # Training Function

In [15]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    # train 모드로 변경
    model.train()

    # for the Mixed Precision
    # Pytorch 예제 : https://pytorch.org/docs/stable/notes/amp_examples.html#amp-examples
    if(FP16):
        scaler = amp.GradScaler()

    dataset_size = 0
    running_loss = 0
    step_acc = 0
    accuracy = 0
    f1 = 0

    bar = tqdm(enumerate(dataloader), total=len(dataloader))

    for step, (img, target) in bar:
        img = img.to(device)
        target = target.to(device)

        batch_size = img.shape[0]

        if(FP16):
            with amp.autocast(enabled=True):
                logits = model(img)
                loss = criterion(logits, target)

                # loss를 Scale
                # Scaled Grdients를 계산(call)하기 위해 scaled loss를 backward()
                scaler.scale(loss).backward()
                # scaler.step() first unscales the gradients of the optimizer's assigned params.
                # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
                # otherwise, optimizer.step() is skipped.
                scaler.step(optimizer)

                # Updates the scale for next iteration.
                scaler.update()

        else:
            logits = model(img)
            loss = criterion(logits, target)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()

        # logits (bs, seq_len, VOCAB_SIZE)
        # trg_output (bs, seq_len)

        # zero the parameter gradients
        optimizer.zero_grad()

        # change learning rate by Scheduler
        if scheduler is not None:
            scheduler.step()

        # loss.item()은 loss를 Python Float으로 반환
        # loss.item()은 batch data의 average loss이므로, sum of loss를 구하기 위해 batch_size를 곱해준다
        running_loss += loss.item() * batch_size
        dataset_size += batch_size
        
        # logits
        logits = logits.detach().cpu()

        # acc, f1
        output = np.argmax(torch.softmax(logits, dim = -1), axis = -1)
        step_acc = np.mean(
            output.view(-1).numpy() == target.view(-1).detach().cpu().numpy())
        step_f1 = f1_score(output.view(-1).numpy(),
                           target.view(-1).detach().cpu().numpy(),average='macro')
        accuracy += step_acc
        f1 += step_f1

        # loss
        train_loss = running_loss / dataset_size

        bar.set_postfix(
            Epoch=epoch, Train_Loss=train_loss, LR=optimizer.param_groups[0]["lr"], accuracy=accuracy / np.float(
                step+1), f1 = f1 / np.float(step+1)
        )

        # break

    accuracy /= len(dataloader)
    f1 /= len(dataloader)
    # Garbage Collector
    gc.collect()

    return train_loss, accuracy, f1


   # Validation Function

In [16]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0
    accuracy = 0
    f1 = 0


    bar = tqdm(enumerate(dataloader), total=len(dataloader))

    for step, (img, target) in bar:
        img = img.to(device)
        target = target.to(device)

        batch_size = img.shape[0]

        logits = model(img)
        loss = criterion(logits, target)

        running_loss += loss.item() * batch_size
        dataset_size += batch_size

        # 실시간으로 정보를 표시하기 위한 epoch loss
        val_loss = running_loss / dataset_size

        # logits
        logits = logits.detach().cpu()
        
        output = np.argmax(torch.softmax(logits, dim=-1), axis=-1)
        step_acc = np.mean(
            output.view(-1).numpy() == target.view(-1).detach().cpu().numpy())
        step_f1 = f1_score(output.view(-1).numpy(),
                           target.view(-1).detach().cpu().numpy(), average='macro')
        accuracy += step_acc
        f1 += step_f1


        bar.set_postfix(
            Epoch=epoch, Valid_Loss=val_loss, LR=optimizer.param_groups[0]["lr"], accuracy=accuracy / np.float(
                step+1), f1 = f1 / np.float(step+1)
        )

        # break

    accuracy /= len(dataloader)
    f1 /= len(dataloader)

    gc.collect()

    return val_loss, accuracy, f1


In [20]:
START_EPOCH = 0


def run_training(
    model,
    optimizer,
    scheduler,
    device,
    num_epochs,
    metric_prefix="",
    file_prefix="",
    early_stopping=True,
    early_stopping_step=10,
):
    # To automatically log graidents
    wandb.watch(model, log_freq=100)

    if torch.cuda.is_available():
        print("[INFO] Using GPU:{}\n".format(torch.cuda.get_device_name()))

    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf
    history = defaultdict(list)
    early_stop_counter = 0
    global START_EPOCH

    # num_epochs만큼, train과 val을 실행한다
    for epoch in range(START_EPOCH, START_EPOCH + num_epochs + 1):
        gc.collect()

        fold_num = 5
        fold = epoch % fold_num

        # for fold in range(fold_num) :

        trainDataloader = train_dataloaders[fold]
        validDataloader = valid_dataloaders[fold]

        # train_train_loss_list = []
        # train_accuracy_list = []
        # train_f1_list = []
        # val_loss_list = []
        # val_accuracy_list= []
        # val_f1_list = []

        train_train_loss, train_accuracy, train_f1 = train_one_epoch(
            model,
            optimizer,
            scheduler,
            dataloader=trainDataloader,
            device=device,
            epoch=epoch,
        )

        val_loss, val_accuracy, val_f1 = valid_one_epoch(
            model, validDataloader, device=device, epoch=epoch
        )

        #     train_train_loss_list.append(train_train_loss)
        #     train_accuracy_list.append(train_accuracy)
        #     train_f1_list.append(train_f1)
        #     val_loss_list.append(val_loss)
        #     val_accuracy_list.append(val_accuracy)
        #     val_f1_list.append(val_f1)

        # train_train_loss = np.mean(train_train_loss_list)
        # train_accuracy = np.mean(train_accuracy_list)
        # train_f1 = np.mean(train_f1_list)
        # val_loss = np.mean(val_loss_list)
        # val_accuracy = np.mean(val_accuracy_list)
        # val_f1 = np.mean(val_f1_list)

        history[f"{metric_prefix}Train Loss"].append(train_train_loss)
        history[f"{metric_prefix}Train Accuracy"].append(train_accuracy)
        history[f"{metric_prefix}Train F1"].append(train_f1)
        history[f"{metric_prefix}Valid Loss"].append(val_loss)
        history[f"{metric_prefix}Valid Accuracy"].append(val_accuracy)
        history[f"{metric_prefix}Valid F1"].append(val_f1)

        # Log the metrics
        wandb.log(
            {
                f"{metric_prefix}Train Loss": train_train_loss,
                f"{metric_prefix}Valid Loss": val_loss,
                f"{metric_prefix}Train Accuracy": train_accuracy,
                f"{metric_prefix}Valid Accuracy": val_accuracy,
                f"{metric_prefix}Train F1": train_f1,
                f"{metric_prefix}Valid F1": val_f1,
            }
        )

        print(f"Valid Loss : {val_loss}")

        torch.save(model.state_dict(), 'last.bin')
        wandb.save('last.bin')

        # deep copy the model
        if val_loss <= best_loss:
            early_stop_counter = 0

            print(
                f"Validation Loss improved( {best_loss} ---> {val_loss}  )"
            )

            # Update Best Loss
            best_loss = val_loss

            # Update Best Model Weight
            # run.summary['Best RMSE'] = best_loss
            best_model_wts = copy.deepcopy(model.state_dict())

            PATH = "{}epoch{:.0f}_Loss{:.4f}.bin".format(
                file_prefix, epoch, best_loss)
            torch.save(model.state_dict(), PATH)
            torch.save(model.state_dict(),
                       f"{file_prefix}best_{epoch}epoch.bin")
            # Save a model file from the current directory
            wandb.save(PATH)

            print(f"Model Saved")

        elif early_stopping:
            early_stop_counter += 1
            if early_stop_counter > early_stopping_step:
                break

        START_EPOCH = epoch + 1
        # break

    end = time.time()
    time_elapsed = end - start
    print(
        "Training complete in {:.0f}h {:.0f}m {:.0f}s".format(
            time_elapsed // 3600,
            (time_elapsed % 3600) // 60,
            (time_elapsed % 3600) % 60,
        )
    )
    print("Best Loss: {:.4f}".format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)

    return model, history


In [ ]:
wandb.restore('epoch37_Loss0.0232.bin', 'jiwon7258/lg/2priu350', root='./')
model.load_state_dict(torch.load('epoch37_Loss0.0232.bin',
                      map_location=CONFIG['device']))


# Fast Start : 10 epoch

In [ ]:
for param in model.backbone.named_parameters():
    param[1].requires_grad = False

In [ ]:
run_training(
    model=model,
    optimizer=optimizer,
    scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer=optimizer, T_max=CONFIG['T_max'], eta_min=CONFIG['min_lr']),
    device=device,
    num_epochs=10,
    metric_prefix="",
    file_prefix="",
    early_stopping=CONFIG['early_stopping'],
    early_stopping_step=CONFIG['early_stopping_step'],
);


[INFO] Using GPU:Tesla P100-PCIE-16GB



100%|██████████| 73/73 [00:42<00:00,  1.71it/s, Epoch=0, LR=8.76e-5, Valid_Loss=2.27, accuracy=0.627, f1=0.389]


Valid Loss : 2.2721614135200063
Validation Loss improved( inf ---> 2.2721614135200063  )
Model Saved


100%|██████████| 73/73 [00:42<00:00,  1.71it/s, Epoch=1, LR=5.67e-5, Valid_Loss=1.94, accuracy=0.664, f1=0.442]


Valid Loss : 1.9445646788466535
Validation Loss improved( 2.2721614135200063 ---> 1.9445646788466535  )
Model Saved


...

Valid Loss : 0.9759780858665162
Validation Loss improved( 1.0173227431354375 ---> 0.9759780858665162  )
Model Saved


100%|██████████| 73/73 [00:42<00:00,  1.72it/s, Epoch=9, LR=7.96e-5, Valid_Loss=0.968, accuracy=0.793, f1=0.625]


Valid Loss : 0.9684999259988433
Validation Loss improved( 0.9759780858665162 ---> 0.9684999259988433  )
Model Saved


100%|██████████| 73/73 [00:42<00:00,  1.72it/s, Epoch=10, LR=4.58e-5, Valid_Loss=0.909, accuracy=0.803, f1=0.633]


Valid Loss : 0.9090046215305295
Validation Loss improved( 0.9684999259988433 ---> 0.9090046215305295  )
Model Saved
Training complete in 0h 49m 13s
Best Loss: 0.9090


In [ ]:
torch.cuda.empty_cache()
import gc; gc.collect()

534

# Reduced LR (1e-5 ~ 1e-6) : ~50epochs

In [ ]:
optimizer = torch.optim.Adam(
    params=model.parameters(), lr=1e-5, weight_decay=CONFIG['weight_decay'])

In [ ]:
for param in model.backbone.named_parameters():
    param[1].requires_grad = True


In [ ]:
START_EPOCH=39

In [ ]:
wandb.restore('epoch37_Loss0.0232.bin', 'jiwon7258/lg/2priu350', root='./')
model.load_state_dict(torch.load('epoch37_Loss0.0232.bin',
                      map_location=CONFIG['device']))


<All keys matched successfully>

In [ ]:
run_training(
    model=model,
    optimizer=optimizer,
    scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer=optimizer, T_max=CONFIG['T_max'], eta_min=CONFIG['min_lr']),
    # scheduler=None,
    device=device,
    num_epochs=30,
    metric_prefix="",
    file_prefix="",
    early_stopping=CONFIG['early_stopping'],
    early_stopping_step=CONFIG['early_stopping_step'],
);


# ~ 100 epoch

In [21]:
START_EPOCH=70

In [22]:
wandb.restore('last.bin', 'jiwon7258/lg/2priu350', root='./')
model.load_state_dict(torch.load('last.bin',
                      map_location=CONFIG['device']))


<All keys matched successfully>

In [23]:
optimizer = torch.optim.Adam(
    params=model.parameters(), lr=1e-6, weight_decay=CONFIG['weight_decay'])


In [24]:
run_training(
    model=model,
    optimizer=optimizer,
    # scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(
    #     optimizer=optimizer, T_max=CONFIG['T_max'], eta_min=CONFIG['min_lr']),
    scheduler=None,
    device=device,
    num_epochs=30,
    metric_prefix="",
    file_prefix="",
    early_stopping=CONFIG['early_stopping'],
    early_stopping_step=CONFIG['early_stopping_step'],
);


[INFO] Using GPU:Tesla P100-PCIE-16GB



100%|██████████| 73/73 [00:40<00:00,  1.82it/s, Epoch=70, LR=1e-6, Valid_Loss=0.00143, accuracy=0.999, f1=0.998]


Valid Loss : 0.0014315247286682134
Validation Loss improved( inf ---> 0.0014315247286682134  )
Model Saved


100%|██████████| 73/73 [00:39<00:00,  1.83it/s, Epoch=71, LR=1e-6, Valid_Loss=0.00344, accuracy=0.999, f1=0.997]


Valid Loss : 0.003441345346502902


...

Valid Loss : 0.00016934559784941347


100%|██████████| 73/73 [00:39<00:00,  1.86it/s, Epoch=87, LR=1e-6, Valid_Loss=0.000213, accuracy=1, f1=1]


Valid Loss : 0.0002129449709511852


 25%|██▍       | 287/1154 [03:14<09:47,  1.48it/s, Epoch=88, LR=1e-6, Train_Loss=0.0131, accuracy=0.997, f1=0.995]


KeyboardInterrupt: ignored

In [25]:
wandb.run.finish()

Train Accuracy,▃▅▄▄▆█▅▅▃▅▁▅▇▅▆▆▅▂
Train F1,▂▄▃▃▆█▅▅▂▅▁▅█▅▆▆▄▃
Train Loss,▅▄▅▄▂▁▂▄▅▂█▃▁▃▁▂▃▄
Valid Accuracy,▅▅▁█▅██▅█▅▅▅██▅▅██
Valid F1,▅▄▁█▆██▆█▆▅▅██▆▅██
Valid Loss,▃▅█▃▄▁▁█▁▃▃▂▂▁▄▂▁▁
Train Accuracy,0.99372
Train F1,0.99136
Train Loss,0.01775
Valid Accuracy,1.0
Valid F1,1.0
